In [19]:
#@title
#UPGRADE SQL LITE in COLAB
#Link to CoLAB - https://colab.research.google.com/drive/1vsf_H57wp_8MZ4zFnFJf918pUHtPhBKV#scrollTo=Dp38Py2BYkGW
!curl https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release | tar xz
%cd sqlite/
!./configure
!make sqlite3.c
%cd /content
!npx degit coleifer/pysqlite3 -f
!cp sqlite/sqlite3.[ch] .
!python setup.py build_static build
!cp build/lib.linux-x86_64-3.7/pysqlite3/_sqlite3.cpython-37m-x86_64-linux-gnu.so \
     /usr/lib/python3.7/lib-dynload/
# then MENU: Runtime > Restart runtime ...

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.7M  100 10.7M    0     0  8593k      0  0:00:01  0:00:01 --:--:-- 8600k
/content/sqlite
checking build system type... x86_64-pc-linux-gnu
checking host system type... x86_64-pc-linux-gnu
checking for gcc... gcc
checking whether the C compiler works... yes
checking for C compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether we are using the GNU C compiler... yes
checking whether gcc accepts -g... yes
checking for gcc option to accept ISO C89... none needed
checking for a sed that does not truncate output... /bin/sed
checking for grep that handles long lines and -e... /bin/grep
checking for egrep... /bin/grep -E
checking for fgrep... /bin/grep -F
checking for ld used by gcc... /usr/bin/ld
checkin

'3.22.0'

In [118]:
import sqlite3
sqlite3.sqlite_version

'3.35.3'

# Question 1

Given a database of the results of an election, find the number of seats won by each party. There are some rules to going about this:


*   There are many constituencies in a state and many candidates who are contesting the election from each constituency.

*   Each candidate belongs to a party.

*   The candidate with the maximum number of votes in a given constituency wins for that constituency.

The output should be in the following format: Party Seats_won
The ordering should be in the order of seats won in descending order.

Expected Output:

**Democratic 2**

**Republic 1** 

In [119]:
# Prepare Query Data
import sqlite3

conn = sqlite3.connect('test.db')
print("Opened database successfully");

conn.execute('''DROP TABLE IF EXISTS candidates;''')
conn.execute('''DROP TABLE IF EXISTS results;''')

conn.execute('''
CREATE TABLE IF NOT EXISTS candidates(id integer
                                    , gender text
                                    , age integer
                                    , party string);''')

conn.execute('''
CREATE TABLE IF NOT EXISTS results(constituency_id integer
                                    , candidate_id integer
                                    , votes integer);''')

# INSERTING VALUES

conn.execute("INSERT INTO candidates VALUES(1, 'M', 55, 'Democratic');")
conn.execute("INSERT INTO candidates VALUES(2, 'M', 51, 'Democratic');")
conn.execute("INSERT INTO candidates VALUES(3, 'F', 62, 'Democratic');")
conn.execute("INSERT INTO candidates VALUES(4, 'M', 60, 'Republic');")
conn.execute("INSERT INTO candidates VALUES(5, 'F', 61, 'Republic');")
conn.execute("INSERT INTO candidates VALUES(6, 'F', 58, 'Republic');")

# INSERTING VALUES

conn.execute("INSERT INTO results VALUES(1, 1, 847529);")
conn.execute("INSERT INTO results VALUES(1, 4, 283409);")
conn.execute("INSERT INTO results VALUES(2, 2, 293841);")
conn.execute("INSERT INTO results VALUES(2, 5, 394385);")
conn.execute("INSERT INTO results VALUES(3, 3, 429084);")
conn.execute("INSERT INTO results VALUES(3, 6, 303890);")

conn.commit()
conn.close()
print("Data Preparation - Done");

Opened database successfully
Data Preparation - Done


In [120]:
# Test Data Load Success

import pandas as pd
# Create your connection.
cnx = sqlite3.connect('test.db')

df_candidates = pd.read_sql_query("SELECT * FROM candidates", cnx)
df_results = pd.read_sql_query("SELECT * FROM results", cnx)

In [121]:
df_candidates

,id,gender,age,party
0,1,M,55,Democratic
1,2,M,51,Democratic
2,3,F,62,Democratic
3,4,M,60,Republic
4,5,F,61,Republic
5,6,F,58,Republic


In [122]:
df_results

,constituency_id,candidate_id,votes
0,1,1,847529
1,1,4,283409
2,2,2,293841
3,2,5,394385
4,3,3,429084
5,3,6,303890


In [123]:
##SQL SOLUTION
query = '''
SELECT party, count(*) as seats_won from (
SELECT c.party
, r.constituency_id
, r.candidate_id
, r.votes
, ROW_NUMBER() OVER (PARTITION BY constituency_id ORDER BY votes desc) as r_num
FROM 
  results as r 
INNER JOIN
  candidates as c
ON c.id = r.candidate_id
)
WHERE r_num = 1
GROUP BY party
'''
df_solution = pd.read_sql_query(query, cnx)
df_solution

,party,seats_won
0,Democratic,2
1,Republic,1


In [124]:
# SOLUTION USING PANDAS LIBRARY
df_temp = df_results.sort_values(by=['constituency_id', 'votes']
                                 , ascending=[True,False])\
    .groupby(['constituency_id']).head(1)

df_final = pd.merge(df_temp[['candidate_id']]
         , df_candidates[['id', 'party']]
         , left_on='candidate_id'
         , right_on='id'
         , how='inner' 
         )[['party', 'candidate_id']].groupby(['party']).agg(
    count_col=pd.NamedAgg(column="candidate_id", aggfunc="count")
)
df_final.columns = ['seats_won']

In [125]:
df_final

,seats_won
party,
Democratic,2
Republic,1


In [ ]:
# LINK TO COLAB PYTHON NOTEBOOK
# https://colab.research.google.com/drive/1vsf_H57wp_8MZ4zFnFJf918pUHtPhBKV#scrollTo=Dp38Py2BYkGW